In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
783,You will be able to tell within the first 30 s...,positive
750,It is enjoyable and fast-paced. <br /><br />Th...,negative
628,THE WATERDANCE (1991) The main character of Th...,negative
700,I saw this picture in 1940 for $.11 and I woul...,positive
493,I was never all that impressed by Night Galler...,positive


In [25]:
df.iloc[4]['review']

'never impressed night gallery one episode stand out br br a tv network executive audition odd act young nerdy boy proceeds make prognostication exec dismisses whole thing flaky waste time prediction come true next morning br br what first seemed parlor act becomes hit show kid s prediction prove consistently accurate br br then one day refuse show facing imminent showtime everyone s wit s end even threatening legal action fulfill contract make daily prediction br br the young boy relents foretells seemingly utopian tomorrow br br after show befuddled executive asks explanation learn complete truth terrifying reveal'

In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
783,able tell within first second film whether wan...,positive
750,enjoyable fast paced br br there way earth act...,negative
628,waterdance main character waterdance played er...,negative
700,saw picture would like secure dvd film greates...,positive
493,never impressed night gallery one episode stan...,positive


In [5]:
df['sentiment'].value_counts()

sentiment
negative    255
positive    245
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
783,able tell within first second film whether wan...,1
750,enjoyable fast paced br br there way earth act...,0
628,waterdance main character waterdance played er...,0
700,saw picture would like secure dvd film greates...,1
493,never impressed night gallery one episode stan...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/NaveenKotyale3/Senti_Analysis.mlflow')
dagshub.init(repo_owner='NaveenKotyale3', repo_name='Senti_Analysis', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

Initialized MLflow to track repo "NaveenKotyale3/Senti_Analysis"

Repository NaveenKotyale3/Senti_Analysis initialized!

2025/12/11 14:00:29 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/aa097197638642df9aa8ef41ad3df1e7', creation_time=1765441829722, experiment_id='0', last_update_time=1765441829722, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [13]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-12-11 14:00:39,813 - INFO - Starting MLflow run...
2025-12-11 14:00:40,375 - INFO - Logging preprocessing parameters...
2025-12-11 14:00:41,377 - INFO - Initializing Logistic Regression model...
2025-12-11 14:00:41,394 - INFO - Fitting the model...
2025-12-11 14:00:41,428 - INFO - Model training complete.
2025-12-11 14:00:41,428 - INFO - Logging model parameters...
2025-12-11 14:00:41,745 - INFO - Making predictions...
2025-12-11 14:00:41,745 - INFO - Calculating evaluation metrics...
2025-12-11 14:00:41,763 - INFO - Logging evaluation metrics...
2025-12-11 14:00:43,116 - INFO - Saving and logging the model...
2025/12/11 14:00:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-12-11 14:00:51,184 - INFO - Model training and logging completed in 10.81 seconds.
2025-12-11 14:00:51,184 - INFO - Accuracy: 0.728
2025-12-11 14:00:51,189 - INFO - Precision: 

🏃 View run languid-asp-252 at: https://dagshub.com/NaveenKotyale3/Senti_Analysis.mlflow/#/experiments/0/runs/50d294c0937e4220ac8d6b139c91da84
🧪 View experiment at: https://dagshub.com/NaveenKotyale3/Senti_Analysis.mlflow/#/experiments/0
